In [1]:
%matplotlib tk
%load_ext memory_profiler
%load_ext line_profiler

In [2]:
import numpy as np
import scipy.io as sio
import time
#from numba import jit, prange, int32
import numba
import scipy.signal as sg
from scipy.stats import pearsonr
import matplotlib.pyplot as pl
import pygmo as po
import seaborn as sns 
import pandas as pd 
import datetime
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
@jit(nopython=True,cache=True)
def KMAOcommu(phi,r,maxDlay,dlayStep,eta,dlt,fs,dt):
    commuOff= np.arange(0,phi.shape[0]) * phi.shape[1]
    pi2     = 2 * np.pi
    eta2    = 0.5 * eta
    kSr     = np.empty((C))
    phiDif  = np.empty((C))
    sumRsp  = np.empty((C))
    sumPHIsp= np.empty((C))
    for n in xrange(maxDlay,phi.shape[1]-1):
        rsum1       = -dlt * r[:,n]
        rpro1       = eta2 * ( 1 - r[:,n]**2 )
        phipro1     = eta2 * (r[:,n]**2 + 1) / r[:,n]
        for s in range(0,phi.shape[0]):
            idD         = n - dlayStep[:,s] + commuOff
            phiDif      = phi.ravel()[idD] - phi[s,n]
            kSr         = kS[:,s] * r.ravel()[idD]
            sumRsp[s]   = np.sum( kSr * np.cos( phiDif ))
            sumPHIsp[s] = np.sum( kSr * np.sin( phiDif ))
        rdt         = rsum1 + rpro1 * sumRsp
        phidt       = omga + phipro1 * sumPHIsp
        # add differntial step
        r[:,n+1]    = r[:,n] + dt*rdt
        phi[:,n+1]  = np.remainder(phi[:,n] + dt*phidt, pi2)
    r       = r[:,maxDlay+1:]     # remove history samples used in the begining
    phi     = phi[:,maxDlay+1:]
    if fs:   # simple downsampling (there may be aliasing)
        r   = r[:,::int(1/(fs*dt))] 
        phi = phi[:,::int(1/(fs*dt))]
    return r * np.exp(1j* phi)

def filterBands(z,fBands,fs,trans):
    nyq         = fs / 2.0
    envCorrs    = np.empty((z.shape[0],z.shape[0],fBands.shape[0]))
    #ff, axf     = pl.subplots(2, 6)
    for ix, freq in enumerate(fBands):
        # Filter frequency bands
        passCut = freq / nyq
        stopCut = [(freq[0] - trans) / nyq, (freq[1] + trans) / nyq]
        coeFil  = sg.iirdesign(passCut, stopCut, gpass, gstop, analog=False, ftype='ellip', output='sos')
        zFilt   = sg.sosfiltfilt(coeFil,np.imag(z),axis=1,padtype='odd')
        #import pdb; pdb.set_trace()
        zEnv    = np.abs(sg.hilbert(zFilt, axis=1))
        # Correlation discarding warmup time
        envCo = np.corrcoef(zEnv[:,int(tMin*fs):], rowvar=True)
        np.fill_diagonal(envCo,0)
        envCorrs[:,:,ix] = envCo
        #plotFilterResp(nyq,coeFil,axf,ix)
    #pl.show()
    return envCorrs

def fitEmpi2sim(simu,empi):
    if simu.shape != empi.shape:
        raise ValueError('Empitrical and functional connectivity have different shape')
        
    C   = simu.shape[0]
    simuProfile = []
    empiProfile = []
    for ix in range(simu.shape[2]):
        simu1        = simu[...,ix]
        empi1        = empi[...,ix]
        simuProfile  = np.append(simuProfile, simu1[np.triu_indices(C,1)])
        empiProfile  = np.append(empiProfile, empi1[np.triu_indices(C,1)])
    ccoef, pval = pearsonr(simuProfile, empiProfile)
    return -1.0 * ccoef

In [ ]:
fr      = 40.0                  # frequency of oscillatros [Hz]
dlt     = 1.0                   # delta parameter (spread) for Lorentzian distribution of frequencies.
omga    = fr * 2*np.pi          # Omega parameter (mean) for lorentzian distribution of frequencies.
tMax    = 5.0                   # maximum integration time
tMin    = 1.0                   # remove the previous simulation time. Only take satable time 
dt      = 1e-4                  # integration time step   
vel     = 7.6                   # transmision velocity of neural impulses [meter/second]
kG      = 25*90.0               # global coupling, couplilng between commiunities.
kL      = 150.0                 # local coupling, coupling inside the commiunities.
gpass   = 0.0025                # ripple amplitude gain in dB in the pass band
gstop   = 40.0                  # ripple amplitude gain in dB in the stop band
envFil  = 0.5                   # cut-off frequency to filter the envelop of the frequency bads
trans   = 1.0                   # width [Hz] of transition interval between stop and pass bands
fs      = 500.0                 # sampling frequency [Hz] of the genrated signal
nyq     = fs / 2                # Nyquist frequency 
pathdata    = '/home/oscar/Documents/MATLAB/Kuramoto/Cabral/' # path to data
nameAnato   = 'AAL_matrices.mat'         # anatomical network
nameEmpi    = 'FC_Profile_Jul13.mat'     # Empirical Functional connectivity
#fBands  = np.array([[2.0,    6.0], # frequency bands for the band-pass filter
#                    [4.0,    8.0],
#                    [6.0,    10.5],
#                    [8.0,    13.0],
#                    [10.5,   21.5],
#                    [13.0,   30.0],
#                    [21.5,   39.0],
#                    [30.0,   48.0],
#                    [39.0,   66.0],
#                    [52.0,   80.0]])


In [ ]:
# load anatomical data
loaddata    = pathdata + nameAnato
anato       = sio.loadmat(loaddata)     # C: structural connectivity, D: distance between areas.
D           = anato['D']                # Distances beween nodes
kS          = anato['C']                # Strucural/anatomical network  
C           = np.shape(D)[1]            # number of nodes/brain areas
eta         = 1.0 / C;                  # proportion of oscillator per community
# load empirical data
loaddata    = pathdata + nameEmpi
empiri      = sio.loadmat(loaddata)     # fBands: frequency bands, FCf:functional connectivity
fBands      = empiri['freqs']           # bandpass filter frequency bands
empiFC      = empiri['FCf']             # empiprical functional connectivity
empiFC      = np.moveaxis(empiFC,0,-1)  # re-arange the axes to the same manner as simulated data
# Delays
dlay        = D / (1000.0 * vel)                # [seconds] correct from mm to m
dlayStep    = np.around(dlay / dt).astype(int)  # delay on steps backwards to be done
maxDlay     = int(np.max(dlayStep))             # number of time steps for the longest delay
# Coupling matrix
kS          = kS / np.mean(kS[~np.identity(C,dtype=bool)])   # normalize structural network to a mean = 1
kS          = kS * kG / C               # Global coupling
np.fill_diagonal(kS,kL)                 # Local coupling
# node's variables
r           = np.empty((C, int(tMax/dt + maxDlay))) # node phase parameter [C, Nsamples to integrate]
phi         = np.empty((C, int(tMax/dt + maxDlay))) # node phase parameter [C, Nsamples to integrate]
# initial conditions as history for the time delays
omegaT              = omga*np.arange(0,maxDlay*dt+dt,dt)
r[:,0:maxDlay+1]    = 0.1 * np.ones((C,maxDlay+1))
phi[:,0:maxDlay+1]  = np.tile(np.remainder(omegaT,2*np.pi),(C,1))
# Design filter for envelops
passCut     = envFil / nyq
stopCut     = (envFil + trans) / nyq
coeFilEn    = sg.iirdesign(passCut, stopCut, gpass, gstop, analog=False, ftype='ellip', output='sos') 

In [3]:
class kaoDynamicsFixKl():
    def __init__(self):
        self.tMax     = 1.0
        self.tMin     = 0.5
        self.fs       = 500.0
        self.omega    = 40.0
        self.dlt      = 1.0
        self.pathData = '/home/oscar/Documents/MATLAB/Kuramoto/Cabral/'
        self.nameDTI  = 'AAL_matrices.mat'         # anatomical network
        self.nameFC   = 'FC_Profile_Jul13.mat'     # Empirical Functional connectivity
        self.dt       = 1e-4
        self._getEmpiricalData()
        self.dim      = 3
        self.log      = np.empty((0,self.dim+1))
        self.log2     = []
        self.counter  = 0
        
    def get_name(self):
        return "kaoDynamicsFixKl"
    
    def get_logs(self):
        return self.log
    
    def get_logs2(self):
        return self.log2
    
    def save_logs(self,log):
        self.log2.append(log)
          
    def _getEmpiricalData(self): 
        # load anatomical data
        loaddata    = self.pathData + self.nameDTI
        dti         = sio.loadmat(loaddata)     # C: structural connectivity, D: distance between areas.
        self.D      = dti['D']                  # Distances beween nodes
        anato       = dti['C']                  # Strucural/anatomical network
        self.C      = np.shape(self.D)[1]       # number of nodes/brain areas
        self.eta    = 1.0 / self.C;             # proportion of oscillator per community
        self.anato  = anato / np.mean(anato[~np.identity(self.C,dtype=bool)])   # normalize structural network to a mean = 1
        # load functional data
        loaddata    = self.pathData + self.nameFC
        empiri      = sio.loadmat(loaddata)     # fBands: frequency bands, FCf:functional connectivity
        self.fBands = empiri['freqs']           # bandpass filter frequency bands
        empiFC      = empiri['FCf']             # empiprical functional connectivity
        self.empiFC = np.moveaxis(empiFC,0,-1)  # re-arange the axes to the same manner as simulated data
    
    def fitness(self,x):
        vel     = x[0]
        kL      = x[1]
        kG      = x[2]
        kS      = self.getAnatoCoupling(kG,kL)
        dlayStep, maxDlay = self.getDelays(vel)
        r, phi  = self._doNodeContainers(maxDlay)
        z       = kaoDynamicsFixKl._KMAOcommu(phi,r,maxDlay,dlayStep,self.eta,self.dlt,self.fs,self.dt,kS,self.omega)
        fit     = self._fitFilterBands(z)
        self.log= np.vstack((self.log,np.array([fit,vel,kL,kG])))
        self.save_logs(np.array([fit,vel,kL,kG]))
        self.counter += 1
        #print(self.log)
        return fit
    
    def getAnatoCoupling(self,kG,kL):
        """Get anatomical network with couplings"""
        kS          = self.anato * kG / self.C      # Global coupling
        np.fill_diagonal(kS,kL)              # Local coupling
        return kS
    
    def getDelays(self,vel):
        """Return maximum delay and delay steps in samples"""
        dlay        = self.D / (1000.0 * vel)                # [seconds] correct from mm to m
        dlayStep    = np.around(dlay / self.dt).astype(int)  # delay on steps backwards to be done
        maxDlay     = int(np.max(dlayStep))                  # number of time steps for the longest delay
        return dlayStep, maxDlay
    
    def get_bounds(self):
        """Boundaries on: velocity, Kl, Kg"""
        return ([1, 1, 1],[25, 1000, 4000])
    
    def _fitFilterBands(self,z):
        nyq   = self.fs / 2.0
        trans = 1.0
        C     = z.shape[0]
        simuProfile = []
        empiProfile = []
        for ix, freq in enumerate(self.fBands):
            # Filter frequency bands
            passCut = freq / nyq
            stopCut = [(freq[0] - trans) / nyq, (freq[1] + trans) / nyq]
            coeFil  = sg.iirdesign(passCut, stopCut, gpass=0.0025, gstop=40.0, analog=False, ftype='ellip', output='sos')
            zFilt   = sg.sosfiltfilt(coeFil, np.imag(z), axis=1, padtype='odd')
            zEnv    = np.abs(sg.hilbert(zFilt, axis=1))
            # Correlation discarding warmup time
            envCo = np.corrcoef(zEnv[:,int(self.tMin*self.fs):], rowvar=True)
            simuProfile  = np.append(simuProfile, envCo[np.triu_indices(C,1)])
            # Profile Empirical data
            empi1        = self.empiFC[...,ix]
            empiProfile  = np.append(empiProfile, empi1[np.triu_indices(C,1)])
        ccoef, pval = pearsonr(simuProfile, empiProfile)
        return np.array([-1 * ccoef])
    
    @jit(nopython=True,cache=True,nogil=True,parallel=False)
    def _KMAOcommu(phi,r,maxDlay,dlayStep,eta,dlt,fs,dt,kS,omga):
        C       = phi.shape[0]
        commuOff = np.arange(0,C) * phi.shape[1]
        pi2     = 2 * np.pi
        eta2    = 0.5 * eta
        kSr     = np.empty((C))
        phiDif  = np.empty((C))
        sumRsp  = np.empty((C))
        sumPHIsp= np.empty((C))
        for n in range(maxDlay,phi.shape[1]-1):
            rsum1       = -dlt * r[:,n]
            rpro1       = eta2 * ( 1 - r[:,n]**2 )
            phipro1     = eta2 * (r[:,n]**2 + 1) / r[:,n]
            for s in range(0,phi.shape[0]):
                idD         = n - dlayStep[:,s] + commuOff
                phiDif      = phi.ravel()[idD] - phi[s,n]
                kSr         = kS[:,s] * r.ravel()[idD]
                sumRsp[s]   = np.sum( kSr * np.cos( phiDif ))
                sumPHIsp[s] = np.sum( kSr * np.sin( phiDif ))
            rdt         = rsum1 + rpro1 * sumRsp
            phidt       = omga + phipro1 * sumPHIsp
            # add differntial step
            r[:,n+1]    = r[:,n] + dt*rdt
            phi[:,n+1]  = np.remainder(phi[:,n] + dt*phidt, pi2)
        r       = r[:,maxDlay+1:]     # remove history samples used in the begining
        phi     = phi[:,maxDlay+1:]
        # simple downsampling (there may be aliasing)
        r   = r[:,::int(1/(fs*dt))] 
        phi = phi[:,::int(1/(fs*dt))]
        return r * np.exp(1j* phi)
    
    def _doNodeContainers(self,maxDlay):      
        # node's variables
        #import pdb; pdb.set_trace()
        r           = np.empty((self.C, int(self.tMax/self.dt + maxDlay))) # node phase parameter [C, Nsamples to integrate]
        phi         = np.empty((self.C, int(self.tMax/self.dt + maxDlay))) # node phase parameter [C, Nsamples to integrate]
        # initial conditions as history for the time delays
        omegaT      = self.omega * np.linspace(0,maxDlay*self.dt+self.dt,maxDlay+1)
        r[:,0:maxDlay+1]    = 0.3 * np.ones((self.C,maxDlay+1))
        phi[:,0:maxDlay+1]  = np.tile(np.remainder(omegaT,2*np.pi),(self.C,1))
        return r, phi

In [ ]:

archi  = po.archipelago()
effe   = np.array([0.3,0.3,0.3,0.5,0.5,0.5,0.9,0.9,0.9])
cross  = np.array([0.3,0.5,0.9,0.3,0.5,0.9,0.3,0.5,0,9])
logsIsl= []
algos  = []
islands= []
for ix in range(0,2):
    effei  = effe[ix]
    crossi = cross[ix]
    algo = po.algorithm(po.de(gen = 5, F=effei, CR=crossi))
    algo.set_verbosity(1)
    algos.append(algo)    
    isl = po.island(algo = algo, prob=prob, size=5,seed=22)
    isl.evolve()
    logsIsl.append(isl.get_algorithm().extract(po.de).get_log())
    islands.append(isl)
    archi2.push_back(algo=algo, prob=prob, size=5, seed=22)
archi2.evolve()
#archiLog = archi2.get_algorithm().extract(po.de).get_log()


In [ ]:
generations = 5
seed = 22
x = [np.array([10,10,100]),np.array([20,20,200]),np.array([5,5,50]),
     np.array([12,220,227]),np.array([8,150,1500])]

# algorithm
algo   = po.algorithm(po.de(gen=generations,seed=seed))
algo.set_verbosity(1)
algo.set_seed(seed)
sizePop= 5
# problem
prob   = po.problem(kaoDynamicsFixKl())
# population
pop    = po.population(prob=prob)
for chromo in x:
    pop.push_back(chromo)
# islands
isl1 = po.island(algo=algo, pop=pop)
algoIs = isl1.get_algorithm()
# evolution
isl1E = isl1.evolve()
#popE = algo.evolve(pop)

In [ ]:
[method_name for method_name in dir(pro) if callable(getattr(pro, method_name))]

In [21]:
#from numba import float64, int64, complex128, prange

class Testkao():
    def __init__(self):
        self.tMax     = np.float32(15.0)
        self.tMin     = np.float32(1.0)
        self.fs       = np.float32(500.0)
        self.omega    = np.float32(2*np.pi * 40.0)
        self.dlt      = np.float32(1.0)
        self.pathData = '/home/oscar/Documents/MATLAB/Kuramoto/Cabral/'
        self.nameDTI  = 'AAL_matrices.mat'         # anatomical network
        self.nameFC   = 'Real_Band_FC.mat'     # Empirical Functional connectivity
        self.dt       = np.float32(1e-4)
        
        self.N        = np.int32(100000)
        
        self._getEmpiricalData()
        self._desingFilterBands()
        self.log      = np.empty((0,6))
        self.doSinCosIx()
        
    def get_mylogs(self):
        return self.log
    
    def get_name(self):
        return "kaoDynamicsFixKl"
    
    def doSinCosIx(self):
        d_pi          = np.linspace(-2 * np.pi, 2 * np.pi, self.N)
        self.d_sin = np.float32(np.sin(d_pi))
        self.d_cos = np.float32(np.cos(d_pi))
        
          
    def _getEmpiricalData(self): 
        # load anatomical data
        loaddata    = self.pathData + self.nameDTI
        dti         = sio.loadmat(loaddata)     # C: structural connectivity, D: distance between areas.
        self.D      = dti['D']                  # Distances beween nodes
        anato       = dti['C']                  # Strucural/anatomical network
        self.C      = np.shape(self.D)[1]       # number of nodes/brain areas
        self.eta    = np.float32(1.0 / self.C)             # proportion of oscillator per community
        self.anato  = anato / np.mean(anato[~np.identity(self.C,dtype=bool)])   # normalize structural network to a mean = 1
        # load functional data
        loaddata    = self.pathData + self.nameFC
        empiri      = sio.loadmat(loaddata)              # fBands: frequency bands, FCf:functional connectivity
        self.fBands = empiri['freq_bands'].astype(np.float32) # bandpass filter frequency bands
        empiFC      = empiri['FC_Env_mean']              # empiprical functional connectivity
        empiFC = np.moveaxis(empiFC,0,-1)                # re-arange the axes to the same manner as simulated data
        self.empiProfile = []
        for ix in range(0,self.fBands.shape[0]):         # Profile Empirical data
            empi1            = empiFC[...,ix]
            self.empiProfile = np.append(self.empiProfile, empi1[np.triu_indices(self.C,1)])
    
    def _desingFilterBands(self):
        nyq   = self.fs / 2.0
        trans = 2.0
        self.coeFil = []
        for freq in self.fBands:
            # Filter frequency bands
            passCut = freq / nyq
            stopCut = [(freq[0] - trans) / nyq, (freq[1] + trans) / nyq]
            self.coeFil.append(sg.iirdesign(passCut, stopCut, gpass=0.0025, gstop=30.0,
                                            analog=False, ftype='cheby2', output='sos'))
        # Filter envelops
        self.coeFilEnv = sg.iirdesign(0.5 / nyq, (0.5+trans)/nyq , gpass=0.0025, gstop=30.0,
                                            analog=False, ftype='cheby2', output='sos')
    
    def _doKuramotoOrder(self, z):
        # global order
        orderG  = np.mean(np.abs( np.mean( z[:,int(self.tMin*self.fs):], axis = 0 )))
        # local order
        orderL  = np.mean(np.mean( np.abs(z[:,int(self.tMin*self.fs):]), axis = 0 ))
        return orderG, orderL     

    def fitness4(self,x):
        vel     = x[0]
        kL      = x[1]
        kG      = x[2]
        kS      = self.getAnatoCoupling(kG,kL)
        dlayStep, maxDlay = self.getDelays(vel)
        r, phi  = self._doNodeContainers(maxDlay)
        #dlayIdx = self.doIndexDelay(r,dlayStep)
        #d_pi          = np.linspace(-2 * np.pi, 2 * np.pi, self.N)
        z    = Testkao._KMAOcommu4(phi,r,maxDlay,dlayStep,self.eta,self.dlt,self.fs,self.dt,kS,self.omega,self.d_sin,self.d_cos)
        self.z = z
        fit, self.envCo = self._fitFilterBands(z)
        orderG, orderL = self._doKuramotoOrder(z)
        self.log= np.vstack((self.log, np.array([fit,vel,kL,kG,orderL,orderG])))
        return np.array([fit])    
    
    def fitness3(self,x):
        vel     = x[0]
        kL      = x[1]
        kG      = x[2]
        kS      = self.getAnatoCoupling(kG,kL)
        dlayStep, maxDlay = self.getDelays(vel)
        r, phi  = self._doNodeContainers(maxDlay)
        dlayIdx = self.doIndexDelay(r,dlayStep)
        d_pi          = np.linspace(-2 * np.pi, 2 * np.pi, self.N)
        z    = Testkao._KMAOcommu3(phi,r,maxDlay,dlayIdx,self.eta,self.dlt,self.fs,self.dt,kS,self.omega,self.d_sin,self.d_cos,d_pi)
        self.z = z
        fit, self.envCo = self._fitFilterBands(z)
        orderG, orderL = self._doKuramotoOrder(z)
        self.log= np.vstack((self.log, np.array([fit,vel,kL,kG,orderL,orderG])))
        return np.array([fit])  
    
    def fitness2(self,x):
        vel     = x[0]
        kL      = x[1]
        kG      = x[2]
        kS      = self.getAnatoCoupling(kG,kL)
        dlayStep, maxDlay = self.getDelays(vel)
        r, phi  = self._doNodeContainers(maxDlay)
        dlayIdx = self.doIndexDelay(r,dlayStep)
        z    = Testkao._KMAOcommu2(phi,r,maxDlay,dlayIdx,self.eta,self.dlt,self.fs,self.dt,kS,self.omega)
        self.z = z
        fit, self.envCo = self._fitFilterBands(z)
        orderG, orderL = self._doKuramotoOrder(z)
        self.log= np.vstack((self.log, np.array([fit,vel,kL,kG,orderL,orderG])))
        return np.array([fit])
    
    def fitness(self,x):
        vel     = x[0]
        kL      = x[1]
        kG      = x[2]
        kS      = self.getAnatoCoupling(kG,kL)
        dlayStep, maxDlay = self.getDelays(vel)
        r, phi  = self._doNodeContainers(maxDlay)
        dlayIdx = self.doIndexDelay2(r,dlayStep)
        z    = Testkao._KMAOcommu(phi,r,maxDlay,dlayIdx,self.eta,self.dlt,self.fs,self.dt,kS,self.omega)
        self.z = z
        fit, self.envCo = self._fitFilterBands(z)
        orderG, orderL = self._doKuramotoOrder(z)
        self.log= np.vstack((self.log, np.array([fit,vel,kL,kG,orderL,orderG])))
        return np.array([fit])
      
    
    def doIndexDelay(self,r,dlayStep):
        commuOff = np.arange(0,r.shape[0]) * r.shape[1]
        commuOff = np.tile(commuOff,(r.shape[0],1)).T
        outpu = dlayStep + commuOff
        return outpu
    
    def doIndexDelay2(self,r,dlayStep):
        nodes   = np.tile(np.arange(self.C),(self.C,1)) * r.shape[1]
        outpu   = nodes - dlayStep
        return outpu
    
    def getAnatoCoupling(self,kG,kL):
        """Get anatomical network with couplings"""
        kS = self.anato * kG / self.C        # Globa  coupling
        np.fill_diagonal(kS,kL)              # Local coupling
        return kS
    
    def getDelays(self,vel):
        """Return maximum delay and delay steps in samples"""
        dlay        = self.D / (1000.0 * vel)                # [seconds] correct from mm to m
        dlayStep    = np.around(dlay / self.dt).astype(np.int32)  # delay on steps backwards to be done
        maxDlay     = np.int32(np.max(dlayStep))                  # number of time steps for the longest delay
        return dlayStep, maxDlay
    
    def get_bounds(self):
        """Boundaries on: velocity, Kl, Kg"""
        return ([0.1, 0, 0],[25, 1000, 5000])
    
    def _fitFilterBands(self,z):
        simuProfile = []
        for coefsos in self.coeFil:
            # filter frequency bands
            zFilt   = sg.sosfiltfilt(coefsos, np.imag(z), axis=1, padtype='odd')
            zEnv    = np.abs(sg.hilbert(zFilt, axis=1))
            # filter envelope
            zEnvFilt= sg.sosfiltfilt(self.coeFilEnv, zEnv, axis=1, padtype='odd')
            # Correlation discarding warmup time
            envCo = np.corrcoef(zEnvFilt[:,int(self.tMin*self.fs):-int(self.tMin*self.fs)], 
                                rowvar=True)
            simuProfile  = np.append(simuProfile, envCo[np.triu_indices(z.shape[0],1)])
        #print(simuProfile.shape)
        ccoef, pval = pearsonr(simuProfile, self.empiProfile)
        return -1 * ccoef, envCo
    #complex128[:,:](float64[:,:],float64[:,:],int64,int64[:,:],float64,float64,float64,float64,float64[:,:],float64), 
    @numba.jit(nopython=True,cache=True,nogil=True,parallel=True,fastmath=False)
    def _KMAOcommu(phi,r,maxDlay,dlayStep,eta,dlt,fs,dt,kS,omga):
        C       = phi.shape[0]
        #nodes   = range(0,C)
        #commuOff = np.arange(0,C) * phi.shape[1]
        pi2     = 2 * np.pi
        eta2    = 0.5 * eta
        sumRsp  = np.empty((C))
        sumPHIsp= np.empty((C))
        for n in range(maxDlay,phi.shape[1]-1):
            rsum1       = -dlt * r[:,n]
            rpro1       = eta2 * ( 1 - r[:,n] * r[:,n])# r*r
            phipro1     = eta2 * (r[:,n] * r[:,n] + 1) / r[:,n]
            idD = n + dlayStep
            #for s in nodes:
            for s in numba.prange(C):
                #idD         = n - dlayStep[:,s] + commuOff
                phiDif      = phi.ravel()[idD[s,:]] - phi[s,n]
                kSr         = kS[:,s] * r.ravel()[idD[s,:]]
                sumRsp[s]   = np.sum( kSr * np.cos( phiDif ))
                sumPHIsp[s] = np.sum( kSr * np.sin( phiDif ))
            rdt         = rsum1 + rpro1 * sumRsp
            phidt       = omga + phipro1 * sumPHIsp
            # add differntial step
            r[:,n+1]    = r[:,n] + dt*rdt
            phi[:,n+1]  = np.remainder(phi[:,n] + dt*phidt, pi2)
        r       = r[:,maxDlay+1:]     # remove history samples used in the begining
        phi     = phi[:,maxDlay+1:]
        # simple downsampling (there may be aliasing)
        r   = r[:,::np.int32(1./(fs*dt))] 
        phi = phi[:,::np.int32(1./(fs*dt))]
        return r * np.exp(1j* phi)

    @numba.jit(nopython=True,cache=True,nogil=True,parallel=True,fastmath=False)
    def _KMAOcommu2(phi,r,maxDlay,dlayStep,eta,dlt,fs,dt,kS,omga):         
        C       = phi.shape[0]
        #nodes   = range(0,C)
        pi2     = 2 * np.pi
        eta2    = 0.5 * eta
        #sumRsp  = np.empty((C))
        #sumPHIsp= np.empty((C))
        
        phiDif  = np.zeros((C,C), dtype=np.float32)
        r_rs    = np.zeros((C,C), dtype=np.float32)
        
        #kSr     = np.zeros((C,C), dtype=np.float32)
        
        for n in range(maxDlay,phi.shape[1]-1):
            rsum1       = -dlt * r[:,n]
            rpro1       = eta2 * ( 1 - r[:,n] * r[:,n])# r*r
            phipro1     = eta2 * (r[:,n] * r[:,n] + 1) / r[:,n]
            idD = n - dlayStep
            #for s in nodes:
            phi_r  = phi.ravel()
            r_r    = r.ravel()
            for s in numba.prange(C):
                #idD         = n - dlayStep[:,s] + commuOff
                phiDif[:,s] = phi_r[idD[:,s]] - phi[s,n]
                r_rs[:,s]  = r_r[idD[:,s]]
                
            kSr      = kS * r_rs
            sumRsp   = np.sum( kSr * np.cos( phiDif ), axis=0)
            sumPHIsp = np.sum( kSr * np.sin( phiDif ), axis=0)
                
            rdt         = rsum1 + rpro1 * sumRsp
            phidt       = omga + phipro1 * sumPHIsp
            # add differntial step
            r[:,n+1]    = r[:,n] + dt*rdt
            phi[:,n+1]  = np.remainder(phi[:,n] + dt*phidt, pi2)
        r       = r[:,maxDlay+1:]     # remove history samples used in the begining
        phi     = phi[:,maxDlay+1:]
        # simple downsampling (there may be aliasing)
        r   = r[:,::np.int32(1./(fs*dt))] 
        phi = phi[:,::np.int32(1./(fs*dt))]
        return r * np.exp(1j* phi)
    
    @numba.jit(nopython=True,cache=True,nogil=True,parallel=True,fastmath=False)
    def _KMAOcommu3(phi,r,maxDlay,dlayStep,eta,dlt,fs,dt,kS,omga,d_sin,d_cos,d_pi):         
        C       = phi.shape[0]  
        
        idPhi  = np.zeros(C, dtype=np.int32)
        
        #nodes   = range(0,C)
        #commuOff = np.arange(0,C) * phi.shape[1]
        pi2     = np.float32(2 * np.pi)
        eta2    = np.float32(0.5 * eta)
        NoverPi2= np.float32(d_cos.shape[0] / pi2)
        sumRsp  = np.empty((C))
        sumPHIsp= np.empty((C))
        for n in range(maxDlay,phi.shape[1]-1):
            rsum1       = -dlt * r[:,n]
            rpro1       = eta2 * ( 1 - r[:,n] * r[:,n])# r*r
            phipro1     = eta2 * (r[:,n] + 1 / r[:,n]) # (r**2 + 1) / r
            idD = n - dlayStep
            #for s in nodes:
            for s in numba.prange(C):
                #idD         = n - dlayStep[:,s] + commuOff
                phiDif      = phi.ravel()[idD[:,s]] - phi[s,n]
                kSr         = kS[:,s] * r.ravel()[idD[:,s]]
                
                idPhi_ = phiDif * NoverPi2
                
                #idPhi = np.searchsorted(d_pi, phiDif)
                
                for i in range(C):
                    idPhi[i] = np.int32(idPhi_[i])
                
                #idPhi = np.rint(phiDif*NoverPi2).astype(np.int32)
                #print(idPhi.shape, d_cos.shape, d_sin.shape)
                
                sumRsp[s]   = np.sum( kSr * d_cos[idPhi]) #np.cos( phiDif ))
                sumPHIsp[s] = np.sum( kSr * d_sin[idPhi])# np.sin( phiDif ))
                
            rdt         = rsum1 + rpro1 * sumRsp
            phidt       = omga + phipro1 * sumPHIsp
            # add differntial step
            r[:,n+1]    = r[:,n] + dt*rdt
            phi[:,n+1]  = np.remainder(phi[:,n] + dt*phidt, pi2)
        r       = r[:,maxDlay+1:]     # remove history samples used in the begining
        phi     = phi[:,maxDlay+1:]
        # simple downsampling (there may be aliasing)
        r   = r[:,::np.int64(1./(fs*dt))] 
        phi = phi[:,::np.int64(1./(fs*dt))]
        return r * np.exp(1j* phi)
    
    
    @numba.jit(nopython=True,cache=True,nogil=True,parallel=True,fastmath=False)
    def _KMAOcommu4(phi,r,maxDlay,dlayStep,eta,dlt,fs,dt,kS,omga,d_sin,d_cos):         
        C       = phi.shape[0]  
        
        idPhi  = np.int32(0)
        #idPhi  = np.zeros(C,dtype=np.int32)
        #kSr    = np.zeros((C))
        
        
        #nodes   = range(0,C)
        #commuOff = np.arange(0,C) * phi.shape[1]
        pi2     = np.float32(2 * np.pi)
        eta2    = np.float32(0.5 * eta)
        NoverPi2= np.float32(d_cos.shape[0] / pi2)
        sumRsp  = np.zeros((C, C),dtype=np.float32)
        sumPHIsp= np.zeros((C, C),dtype=np.float32)
        for n in range(maxDlay,phi.shape[1]-1):
            rsum1       = -dlt * r[:,n]
            rpro1       = eta2 * ( 1 - r[:,n] * r[:,n])# r*r
            phipro1     = eta2 * (r[:,n] + 1 / r[:,n]) # (r**2 + 1) / r
            idD = n - dlayStep
            #for s in nodes:
            for s in numba.prange(C):
                for i in range(C):
                    phiDif      = phi[s,idD[s,i]] - phi[s,n]
                    kSr         = kS[s,i] * r[s,idD[s,i]]
                    idPhi       = np.int32(phiDif * NoverPi2)
                    sumRsp[s,i]   = kSr * d_cos[idPhi]
                    sumPHIsp[s,i] = kSr * d_sin[idPhi]

            sumRsp_   = np.sum(sumRsp, axis=1)
            sumPHIsp_ = np.sum(sumPHIsp, axis=1)
            
            rdt         = rsum1 + rpro1 * sumRsp_
            phidt       = omga + phipro1 * sumPHIsp_
            # add differntial step
            r[:,n+1]    = r[:,n] + dt*rdt
            phi[:,n+1]  = np.remainder(phi[:,n] + dt*phidt, pi2)
        r       = r[:,maxDlay+1:]     # remove history samples used in the begining
        phi     = phi[:,maxDlay+1:]
        # simple downsampling (there may be aliasing)
        r   = r[:,::np.int64(1./(fs*dt))] 
        phi = phi[:,::np.int64(1./(fs*dt))]
        return r * np.exp(1j* phi)
        
    def _doNodeContainers(self,maxDlay):      
        # node's variables
        #import pdb; pdb.set_trace()
        r           = np.empty((self.C, int(self.tMax/self.dt + maxDlay)),dtype=np.float32) # node phase parameter [C, Nsamples to integrate]
        phi         = np.empty((self.C, int(self.tMax/self.dt + maxDlay)),dtype=np.float32) # node phase parameter [C, Nsamples to integrate]
        # initial conditions as history for the time delays
        omegaT      = self.omega * np.linspace(0,maxDlay*self.dt+self.dt,maxDlay+1)
        r[:,0:maxDlay+1]    = 0.3 * np.ones((self.C,maxDlay+1))
        #phi[:,0:maxDlay+1]  = np.tile(np.remainder(omegaT,2*np.pi),(self.C,1))
        phi[:,0:maxDlay+1]  = 0.3 * np.ones((self.C,maxDlay+1))
        #print(r.dtype, phi.dtype)
        return r, phi
    
    
    def setVariablesModel(self,x):
        "set the basic variable need to run the KM model"
        vel     = x[0]
        kL      = x[1]
        kG      = x[2]
        self.kS      = self.getAnatoCoupling(kG,kL)
        self.dlayStep, self.maxDlay = self.getDelays(vel)
        self.r, self.phi   = self._doNodeContainers(self.maxDlay)
        self.dlayId  = self.doIndexDelay(self.phi,self.dlayStep)
        #self.doSinCosIx()

/home/oscar/anaconda2/envs/ipy36/lib/python3.6/site-packages/numba/decorators.py:146: RuntimeWarning: Caching is not available when the 'parallel' target is in use. Caching is now being disabled to allow execution to continue.
  warnings.warn(msg, RuntimeWarning)


In [236]:
kao = Testkao()
fit = kao.fitness([0.9,150,1200])
print(fit)
#pl.figure()
pl.plot(np.abs(np.mean(kao.z,axis=0)))
pl.plot(np.mean(np.abs(kao.z),axis=0))
pl.show()

[nan]


/home/oscar/anaconda2/envs/ipy36/lib/python3.6/site-packages/numpy/lib/function_base.py:1128: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/oscar/anaconda2/envs/ipy36/lib/python3.6/site-packages/numpy/core/_methods.py:73: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
/home/oscar/anaconda2/envs/ipy36/lib/python3.6/site-packages/numpy/lib/function_base.py:3175: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/home/oscar/anaconda2/envs/ipy36/lib/python3.6/site-packages/numpy/lib/function_base.py:3109: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)
/home/oscar/anaconda2/envs/ipy36/lib/python3.6/site-packages/numpy/lib/function_base.py:3109: RuntimeWarning: invalid value encountered in multiply
  c *= 1. / np.float64(fact)
/home/oscar/anaconda2/envs/ipy36/lib/python3.6/site-packages/scipy/stats/stats.py:5240: RuntimeWarning: invalid value e

In [19]:
kmt = Testkao()
kmt.setVariablesModel([0.9,150, 1200])

%lprun -f Testkao._KMAOcommu2 Testkao._KMAOcommu2(kmt.phi,kmt.r,kmt.maxDlay,kmt.dlayStep,kmt.eta,kmt.dlt,kmt.fs,kmt.dt,kmt.kS,kmt.omega)

In [11]:
kao2 = Testkao()
#kao4.fitness4([0.9,150,1200])
%timeit kao2.fitness2([0.9,150,1200])

4.99 s ± 215 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
kao = Testkao()
#kao.fitness([0.9,150,1200])
%timeit kao.fitness([0.9,150,1200])

4.25 s ± 460 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
print(kao.log[0,4], kao.log[0,5])
print(kao4.log[0,4], kao4.log[0,5])


0.5337403578698402 0.26626207807380836
0.05760614440176525 0.023579573431403697


In [25]:
N = 1000000
#phi = np.random.uniform(0.00001, 2*np.pi,N)
phi = np.pi/3.2

d_pi     = np.linspace(0,2 * np.pi,N)
d_pi_sin = np.sin(d_pi)
d_pi_cos = np.cos(d_pi)

idpi = np.intp(phi*N/(2*np.pi))

d_phisin = d_pi_sin[idpi]
d_phicos = d_pi_cos[idpi]

#seaSin = np.searchsorted(np.arange(N), phi*N/(2*np.pi))
seaSin = np.searchsorted(d_pi, phi)

print(np.sin(phi))
print(d_phisin)
print(d_pi_sin[seaSin])

print(d_phicos)
print(np.cos(phi))

def doIxsin(x):
    a = 0
    for i in range(N):
        a += d_pi_sin[x[i]]
    return a
        
def doSin(x):
    a = 0
    for i in range(N):
        a += np.sin(x[i])
    return a
        
#print(np.sum(d_phisin))
#print(np.sum(np.sin(phi)))
#print(np.sum(np.abs(d_phisin - np.sin(phi))))
      
#print(np.sum(d_phicos))
#print(np.sum(np.cos(phi)))
#print(np.sum(np.abs(d_phicos - np.cos(phi))))

0.8314696123025452
0.8314701577324907
0.8314701577324907
0.5555694167251353
0.5555702330196023


In [43]:
%timeit doIxsin(idpi)
print(doIxsin(idpi))

255 ms ± 5.03 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
118.96799265821375


In [42]:
%timeit doSin(phi)
print(doSin(phi))

1.4 s ± 33.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
118.97003055298563


In [61]:
N = np.int32(3000)
real =  np.random.rand(N, 100).astype(np.float32)
imag =  np.random.rand(N, 100).astype(np.float32)

z    = (real + 1j * imag)

@numba.jit(nopython=True, nogil=False, parallel=False)
def doTestNumba(z,N):
    ordeiG = np.zeros(N,dtype=np.float32)
    ordeiL = np.zeros(N,dtype=np.float32)
    # global order
    for i in range(N):
        ordeiG[i] = np.absolute(np.mean(z[i,:]))
        ordeiL[i] = np.mean( np.absolute(z[i,:]))
        
    orderG  = np.mean( ordeiG)
    orderGst  = np.std( ordeiG)
    # local order
    orderL  = np.mean(ordeiL)
    orderLst  = np.std(ordeiL)
    return orderG, orderL, orderGst, orderLst  

def doTest(z):
    # global order
    orG = np.abs( np.mean( z, axis = 0 ))
    orderG  = np.mean(orG)
    orderGst  = np.std(orG)
    # local order
    orL = np.mean( np.abs(z), axis = 0 )
    orderL  = np.mean(orL)
    orderLst  = np.std(orL)
    return orderG, orderL, orderGst, orderLst 


def testDelays(vel,D,dt):
    """Return maximum delay and delay steps in samples"""
    dlay        = D / (1000.0 * vel)                # [seconds] correct from mm to m
    dlayStep    = np.around(dlay / dt).astype(np.int32)  # delay on steps backwards to be done
    maxDlay     = np.int32(np.max(dlayStep))                  # number of time steps for the longest delay
    return dlayStep, maxDlay

#@numba.jit(nopython=True)
def testDelaysNumba(vel,D,dt):
    """Return maximum delay and delay steps in samples"""
    dlay        = D / (1000.0 * vel)                # [seconds] correct from mm to m
    dlayStep    = (dlay / dt).astype(np.int32)  # delay on steps backwards to be done
    maxDlay     = np.int32(np.max(dlayStep))                  # number of time steps for the longest delay
    return dlayStep, maxDlay


%timeit testDelays(5.37, kao.D, kao.dt)
%timeit testDelaysNumba(5.37, kao.D, kao.dt)

#%timeit doTestNumba(z,N)
#%timeit doTest(z)

41.3 µs ± 285 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
23.1 µs ± 204 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [4]:
Nout = 100000
Nin  = 100
x    = np.arange(Nin, dtype=np.float32)

@numba.jit(nopython=True, nogil=False, parallel=False)
def testAsType(x,Nout,Nin):
    for i in range(Nout):
        for j in range(Nin): 
            y = x.astype(np.int32)
            
@numba.jit(nopython=True, nogil=False, parallel=False)
def testNoType(x,Nout,Nin):
    for i in range(Nout):
        for j in range(Nin): 
            y = x

@numba.jit(nopython=True, nogil=True, parallel=True)
def testAsTypeParallel(x,Nout,Nin):
    for i in range(Nout):
        for j in numba.prange(Nin): 
            y = x.astype(np.int32)
            
@numba.jit(nopython=True, nogil=True, parallel=True)
def testNoTypeParallel(x,Nout,Nin):
    for i in range(Nout):
        for j in numba.prange(Nin): 
            y = x
            
            
%timeit testAsType(x,Nout,Nin)
%timeit testNoType(x,Nout,Nin)
%timeit testAsTypeParallel(x,Nout,Nin)
%timeit testNoTypeParallel(x,Nout,Nin)


1.05 s ± 19.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
329 ns ± 7.24 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.71 s ± 6.13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
427 ns ± 5.84 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [143]:
#coeFil, coeFilEnv = desingFilterBandsBA()

nyq   = kao.fs / 2.0
trans = 2.0
freq  = kao.fBands[0]
passCut = freq / nyq
stopCut = [(freq[0] - trans) / nyq, (freq[1] + trans) / nyq]

#b, a    = sg.iirdesign(passCut[0], stopCut[0], gpass=0.0025, gstop=40.0, analog=False, ftype='ellip', output='ba')
#zFilt   = sg.filtfilt(b, a, np.imag(kao.z), axis=1, method='pad',padlen=1000)#, padtype='odd')
#b, a    = sg.iirdesign(passCut[1], stopCut[1], gpass=0.0025, gstop=40.0, analog=False, ftype='ellip', output='ba')
#zFilt   = sg.filtfilt(b, a, zFilt, axis=1, method='pad',padlen=1000)#, padtype='odd')

b, a    = sg.iirdesign(passCut, stopCut, gpass=0.0025, gstop=30.0, analog=False, ftype='ellip', output='ba')
zFilt   = sg.filtfilt(b, a, np.imag(kao.z), axis=1, method='pad',padlen=500)#, padtype='odd')
#plotSingleFilterResp(nyq,(b, a))
zEnv    = np.abs(sg.hilbert(zFilt, axis=1))


#b, a    = sg.iirdesign(passCut, stopCut, gpass=0.0025, gstop=30.0, analog=False, ftype='cheby2', output='ba')
#zFilt2   = sg.filtfilt(b, a, np.imag(kao.z), axis=1, method='pad',padlen=1000)#, padtype='odd')
#plotSingleFilterResp(nyq,(b, a))
#zEnv2    = np.abs(sg.hilbert(zFilt2, axis=1))


#sos    = sg.iirdesign(passCut[0], stopCut[0], gpass=0.0025, gstop=40.0, analog=False, ftype='cheby2', output='sos')
#zFilt  = sg.sosfiltfilt(sos, np.imag(kao.z), axis=1, padtype='odd',padlen=1000)
#sos    = sg.iirdesign(passCut[1], stopCut[1], gpass=0.0025, gstop=40.0, analog=False, ftype='cheby2', output='sos')
#zFilt2  = sg.sosfiltfilt(sos, zFilt, axis=1, padtype='odd',padlen=1000)

sos    = sg.iirdesign(passCut, stopCut, gpass=0.0025, gstop=30.0, analog=False, ftype='ellip', output='sos')
zFilt3  = sg.sosfiltfilt(sos, np.imag(kao.z), axis=1, padtype='odd',padlen=500)
#plotSingleFilterResp(nyq,sos)
zEnv3    = np.abs(sg.hilbert(zFilt3, axis=1))


sos    = sg.iirdesign(passCut, stopCut, gpass=0.0025, gstop=30.0, analog=False, ftype='cheby2', output='sos')
zFilt4  = sg.sosfiltfilt(sos, np.imag(kao.z), axis=1, padtype='odd')
#zFilt44 = np.pad(zFilt4, [(0, 0),(1000, 1000)], 'constant', constant_values= (0,0))
zFilt44 = np.pad(zFilt4, [(0, 0),(1000, 1000)], 'symmetric')

#plotSingleFilterResp(nyq,sos)
zEnv4     = np.abs(sg.hilbert(zFilt4, axis=1))
#zEnv4     = np.pad(zEnv4, [(0, 0),(1000, 1000)], 'constant', constant_values= (0,0))
zEnv44    = np.abs(sg.hilbert(zFilt44, axis=1))

sos = sg.iirdesign(0.5 / nyq, (0.5+trans-1)/nyq , gpass=0.0025, gstop=40.0,
                                            analog=False, ftype='ellip', output='sos')
plotSingleFilterResp(nyq,sos)
zEnvFilt4= sg.sosfiltfilt(sos, zEnv4, axis=1, padtype='odd')

sos = sg.iirdesign(0.5 / nyq, (0.5+trans)/nyq , gpass=0.0025, gstop=30.0,
                                            analog=False, ftype='cheby2', output='sos')
plotSingleFilterResp(nyq,sos)
zEnvFilt44= sg.sosfiltfilt(sos, zEnv4, axis=1, padtype='odd')

#pl.figure()
#pl.plot(zEnv[1,:])

#pl.plot(zEnv3[1,:])
#pl.plot(zEnv4[1,:])
#pl.plot(zEnv44[1,:])
#pl.show()

#pl.figure()
#pl.plot(zFilt[1,:])
#pl.plot(zFilt3[1,:])
#pl.plot(zFilt4[1,:])
#pl.plot(zFilt44[1,:])
#pl.show()

pl.figure()
pl.plot(zEnvFilt4[10,:],color='b')
pl.plot(zEnvFilt44[10,:],color='r')
pl.plot(zEnv4[10,:],color='k')
#pl.plot(zEnv44[1,:],color='r')
pl.show()

In [90]:
%timeit sg.sosfiltfilt(sos, np.imag(kao.z), axis=1, padtype='odd',padlen=500)

%timeit sg.sosfiltfilt(sos, np.imag(kao.z), axis=1, padtype='odd',padlen=500)

54.7 ms ± 1.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
54.4 ms ± 532 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [96]:

def desingFilterBandsSOS():
    nyq   = kao.fs / 2.0
    trans = 1.0
    coeFil = []
    for freq in kao.fBands:
        # Filter frequency bands
        passCut = freq / nyq
        stopCut = [(freq[0] - trans+1e-16) / nyq, (freq[1] + trans+1e-16) / nyq]
        coeFil.append(sg.iirdesign(passCut, stopCut, gpass=0.0025, gstop=40.0,
                                        analog=False, ftype='ellip', output='sos'))
    # Filter envelops
    coeFilEnv = sg.iirdesign(0.5 / nyq, (0.5+trans)/nyq , gpass=0.0025, gstop=40.0,
                                            analog=False, ftype='ellip', output='sos')
    return coeFil, coeFilEnv
    
def desingFilterBandsBA():
    nyq   = kao.fs / 2.0
    trans = 1.0
    coeFil = []
    for freq in kao.fBands:
        # Filter frequency bands
        passCut = freq / nyq
        stopCut = [(freq[0] - trans+1e-16) / nyq, (freq[1] + trans+1e-16) / nyq]
        coeFil.append(sg.iirdesign(passCut, stopCut, gpass=0.0025, gstop=40.0,
                                        analog=False, ftype='ellip', output='ba'))
    # Filter envelops
    coeFilEnv = sg.iirdesign(0.5 / nyq, (0.5+trans)/nyq , gpass=0.0025, gstop=40.0,
                                            analog=False, ftype='ellip', output='ba')
    return coeFil, coeFilEnv
    

In [3]:
generations = 3
sizePop= 5
# algorithm
algo   = po.algorithm(po.de(gen=generations))
algo.set_verbosity(1)
# problem
prob   = po.problem(myUDP.TestkaoFile())
# population
pop    = po.population(prob=prob, size=sizePop)
# islad
isl = po.island(algo=algo, pop=pop)
# evolution
popEvo = algo.evolve(pop)
isl.evolve()

/home/oscar/anaconda2/envs/ipy36/lib/python3.6/site-packages/scipy/signal/filter_design.py:3399: RuntimeWarning: divide by zero encountered in true_divide
  (stopb * (passb[0] - passb[1])))


In [ ]:
probE  = popE.problem.extract(Testkao)
logged = probE.get_logs()
loguda = algo.extract(po.de).get_log()
print(logged.shape)
print(popE.problem.get_fevals())
print(len(loguda))
print(loguda[0])
print(loguda[-1])


Save text file with the most relevant information of the simulation

In [ ]:
pathsave = '/home/oscar/Documents/PythonProjects/kuramotoAO/optimizationResults/'
filename = 'deOnePopFit022.txt'
bestFstr  = 'champion fitness: ' + str(popE.champion_f[0]) + '; best fit possible: -1'
bestChamp  = 'champion decission vector'
bestXstr  = 'velocity: ' + str(popE.champion_x[0]) + ', kL:' + str(popE.champion_x[1]),', kG: ' + str(popE.champion_x[2])
popStr   = popE.__str__()
algoStr  = algo.__str__()
dateStr  = str(datetime.datetime.now())
with open(pathsave+filename, "w") as text_file:
    print(dateStr, end='\n\n', file=text_file)
    print(bestFstr, end='\n', file=text_file)  
    print(bestChamp, end='\n', file=text_file)
    print(bestXstr, end='\n\n', file=text_file)  
    print(algoStr, end='\n', file=text_file)
    print(popStr, end='\n', file=text_file)
print('Done')

Save numpy array with output parameters

In [ ]:
loguda = algo.extract(po.de).get_log()
# get best fitness per generation
best = np.array([loguda[i][2] for i in range(len(loguda))])
# get parameter for the logging variable in problem class
probE  = popE.problem.extract(Testkao)
logged = probE.get_logs()
fitness= logged[:,0]
velocity=logged[:,1]
kL      = logged[:,2]
kG      = logged[:,3]
#save file
filename = 'deOnePopFit022.npz'
outfile  = pathsave + filename
np.savez(outfile, fitness=fitness, velocity=velocity, kL=kL, kG=kG)

# Test performance

In [39]:
ns = 100
N = 500
x = np.ones((ns,ns),order='C')
t_r = np.ones((N,ns))
t_c = np.ones((ns,N))
idx = np.random.randint(0,ns,ns)

def testRow(x,N,ns):
    #y = np.ones((ns,ns),order='C')
    for i in range(N):
        for j in range(ns):
            y = x[j,:] * t_r[i,:]

def testCol(x,N,ns):
    #y = np.ones((ns,ns),order='C')
    for i in range(N):
        for j in range(ns):
            y = x[:,j] *  t_c[:,i]
            
%timeit testRow(x,N,ns)

%timeit testCol(x,N,ns)

81.3 ms ± 2.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
140 ms ± 802 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Test with time along columns

In [15]:
#from numba import float64, int64, complex128, prange

class TestkaoCol():
    def __init__(self):
        self.tMax     = np.float64(5.0)
        self.tMin     = np.float64(1.0)
        self.fs       = np.float64(500.0)
        self.omega    = np.float64(2*np.pi * 40.0)
        self.dlt      = np.float64(1.0)
        self.pathData = '/home/oscar/Documents/MATLAB/Kuramoto/Cabral/'
        self.nameDTI  = 'AAL_matrices.mat'         # anatomical network
        self.nameFC   = 'Real_Band_FC.mat'     # Empirical Functional connectivity
        self.dt       = np.float64(1e-4)
        
        self.N        = np.int32(100000)
        
        self._getEmpiricalData()
        self._desingFilterBands()
        self.log      = np.empty((0,6),dtype=np.float32)
        self.doSinCosIx()
        
    def get_mylogs(self):
        return self.log
    
    def get_name(self):
        return "kaoDynamicsFixKl"
    
    def doSinCosIx(self):
        d_pi          = np.linspace(-2 * np.pi, 2 * np.pi, self.N)
        self.d_sin = np.float32(np.sin(d_pi))
        self.d_cos = np.float32(np.cos(d_pi))
        
          
    def _getEmpiricalData(self): 
        # load anatomical data
        loaddata    = self.pathData + self.nameDTI
        dti         = sio.loadmat(loaddata)     # C: structural connectivity, D: distance between areas.
        self.D      = dti['D']                  # Distances beween nodes
        anato       = dti['C']                  # Strucural/anatomical network
        self.C      = np.shape(self.D)[1]       # number of nodes/brain areas
        self.eta    = np.float64(1.0 / self.C)             # proportion of oscillator per community
        self.anato  = anato / np.mean(anato[~np.identity(self.C,dtype=bool)])   # normalize structural network to a mean = 1
        # load functional data
        loaddata    = self.pathData + self.nameFC
        empiri      = sio.loadmat(loaddata)              # fBands: frequency bands, FCf:functional connectivity
        self.fBands = empiri['freq_bands'].astype(np.float32) # bandpass filter frequency bands
        empiFC      = empiri['FC_Env_mean']              # empiprical functional connectivity
        empiFC = np.moveaxis(empiFC,0,-1)                # re-arange the axes to the same manner as simulated data
        self.empiProfile = []
        for ix in range(0,self.fBands.shape[0]):         # Profile Empirical data
            empi1            = empiFC[...,ix]
            self.empiProfile = np.append(self.empiProfile, empi1[np.triu_indices(self.C,1)])
    
    def _desingFilterBands(self):
        nyq   = self.fs / 2.0
        trans = 2.0
        self.coeFil = []
        for freq in self.fBands:
            # Filter frequency bands
            passCut = freq / nyq
            stopCut = [(freq[0] - trans) / nyq, (freq[1] + trans) / nyq]
            self.coeFil.append(sg.iirdesign(passCut, stopCut, gpass=0.0025, gstop=30.0,
                                            analog=False, ftype='cheby2', output='sos'))
        # Filter envelops
        self.coeFilEnv = sg.iirdesign(0.5 / nyq, (0.5+trans)/nyq , gpass=0.0025, gstop=30.0,
                                            analog=False, ftype='cheby2', output='sos')
    
    def _doKuramotoOrder(self, z):
        # global order
        orderG  = np.mean(np.abs( np.mean( z[:,int(self.tMin*self.fs):], axis = 1 )))
        # local order
        orderL  = np.mean(np.mean( np.abs(z[:,int(self.tMin*self.fs):]), axis = 1 ))
        return orderG, orderL     

    def fitness4(self,x):
        vel     = x[0]
        kL      = x[1]
        kG      = x[2]
        kS      = self.getAnatoCoupling(kG,kL)
        dlayStep, maxDlay = self.getDelays(vel)
        r, phi  = self._doNodeContainers(maxDlay)
        #dlayIdx = self.doIndexDelay(r,dlayStep)
        #d_pi          = np.linspace(-2 * np.pi, 2 * np.pi, self.N)
        z    = TestkaoCol._KMAOcommu4(phi,r,maxDlay,dlayStep,self.eta,self.dlt,self.fs,self.dt,kS,self.omega,self.d_sin,self.d_cos)
        self.z = z
        fit, self.envCo = self._fitFilterBands(z.T)
        orderG, orderL = self._doKuramotoOrder(z.T)
        self.log= np.vstack((self.log, np.array([fit,vel,kL,kG,orderL,orderG])))
        return np.array([fit])  
    
    
    def fitness2(self,x):
        vel     = x[0]
        kL      = x[1]
        kG      = x[2]
        kS      = self.getAnatoCoupling(kG,kL)
        dlayStep, maxDlay = self.getDelays(vel)
        r, phi  = self._doNodeContainers(maxDlay)
        dlayIdx = self.doIndexDelay(r,dlayStep)
        z    = TestkaoCol._KMAOcommu2(phi,r,maxDlay,dlayIdx,self.eta,self.dlt,self.fs,self.dt,kS,self.omega)
        self.z = z
        fit, self.envCo = self._fitFilterBands(z)
        orderG, orderL = self._doKuramotoOrder(z)
        self.log= np.vstack((self.log, np.array([fit,vel,kL,kG,orderL,orderG])))
        return np.array([fit])
    
    def fitness(self,x):
        vel     = x[0]
        kL      = x[1]
        kG      = x[2]
        kS      = self.getAnatoCoupling(kG,kL)
        self.dlayStep, maxDlay = self.getDelays(vel)
        r, phi  = self._doNodeContainers(maxDlay)
        dlayIdx = self.doIndexDelay2(self.dlayStep)
        z    = TestkaoCol._KMAOcommu(phi,r,maxDlay,dlayIdx,self.eta,self.dlt,self.fs,self.dt,kS,self.omega)
        #print(z.shape)
        self.z = z
        fit, self.envCo = self._fitFilterBands(z.T)
        orderG, orderL = self._doKuramotoOrder(z.T)
        self.log= np.vstack((self.log, np.array([fit,vel,kL,kG,orderL,orderG])))
        return np.array([fit])
    
        
    def doIndexDelay(self,r,dlayStep):
        commuOff = np.arange(0,r.shape[1]) * r.shape[0]
        commuOff = np.tile(commuOff,(r.shape[1],1)).T
        outpu = dlayStep + commuOff
        return outpu
    
    def doIndexDelay2(self,dlayStep):
        '''formula to transform from fancy indexing to flattened array.
            idxDlay must be used as idD[s,:] = n * self.C - idxDlay
        '''
        nodes   = np.tile(np.arange(self.C),(self.C,1))
        idxDlay = self.C * dlayStep + nodes 
        return np.int64(idxDlay)
    
    def getAnatoCoupling(self,kG,kL):
        """Get anatomical network with couplings"""
        kS = self.anato * kG / self.C        # Globa  coupling
        np.fill_diagonal(kS,kL)              # Local coupling
        return np.float64(kS)
    
    def getDelays(self,vel):
        """Return maximum delay and delay steps in samples"""
        dlay        = self.D / (1000.0 * vel)                # [seconds] correct from mm to m
        dlayStep    = np.around(dlay / self.dt).astype(np.int32)  # delay on steps backwards to be done
        maxDlay     = np.int32(np.max(dlayStep))                  # number of time steps for the longest delay
        return dlayStep, maxDlay
    
    def get_bounds(self):
        """Boundaries on: velocity, Kl, Kg"""
        return ([0.1, 0, 0],[25, 1000, 5000])
    
    def _fitFilterBands(self,z):
        #print(z.shape)
        simuProfile = []
        for coefsos in self.coeFil:
            # filter frequency bands
            zFilt   = sg.sosfiltfilt(coefsos, np.imag(z), axis=1, padtype='odd')
            zEnv    = np.abs(sg.hilbert(zFilt, axis=1))
            # filter envelope
            zEnvFilt= sg.sosfiltfilt(self.coeFilEnv, zEnv, axis=1, padtype='odd')
            # Correlation discarding warmup time
            envCo = np.corrcoef(zEnvFilt[:,int(self.tMin*self.fs):-int(self.tMin*self.fs/2)], 
                                rowvar=True)
            simuProfile  = np.append(simuProfile, envCo[np.triu_indices(z.shape[0],1)])
        #print(simuProfile.shape)
        ccoef, pval = pearsonr(simuProfile, self.empiProfile)
        return -1 * ccoef, envCo

    def _fitFilterBands2(self,z):
        #print(z.shape)
        simuProfile = []
        for coefsos in self.coeFil:
            # filter frequency bands
            zFilt   = sg.sosfiltfilt(coefsos, np.imag(z), axis=0, padtype='odd')
            zEnv    = np.abs(sg.hilbert(zFilt, axis=0))
            # filter envelope
            zEnvFilt= sg.sosfiltfilt(self.coeFilEnv, zEnv, axis=0, padtype='odd')
            # Correlation discarding warmup time
            envCo = np.corrcoef(zEnvFilt[int(self.tMin*self.fs):-int(self.tMin*self.fs/2),:], 
                                rowvar=False)
            simuProfile  = np.append(simuProfile, envCo[np.triu_indices(z.shape[1],1)])
        #print(simuProfile.shape)
        ccoef, pval = pearsonr(simuProfile, self.empiProfile)
        return -1 * ccoef, envCo
    
    @numba.jit(nopython=True,cache=True,nogil=True,parallel=True,fastmath=False)
    def _KMAOcommu2(phi,r,maxDlay,dlayStep,eta,dlt,fs,dt,kS,omga):         
        C       = phi.shape[1]
        #nodes   = range(0,C)
        pi2     = np.float32(2 * np.pi)
        eta2    = np.float32(0.5 * eta)
        #sumRsp  = np.empty((C))
        #sumPHIsp= np.empty((C))
        
        phiDif  = np.zeros((C,C), dtype=np.float32)
        r_rs    = np.zeros((C,C), dtype=np.float32)
        
        #kSr     = np.zeros((C,C), dtype=np.float32)
        
        for n in range(maxDlay,phi.shape[0]-1):
            rsum1       = -dlt * r[n,:]
            rpro1       = eta2 * ( 1 - r[n,:] * r[n,:])# r*r
            phipro1     = eta2 * (r[n,:] + 1 / r[n,:])
            idD = n - dlayStep
            #for s in nodes:
            phi_r  = phi.ravel()
            r_r    = r.ravel()
            for s in numba.prange(C):
                #idD         = n - dlayStep[:,s] + commuOff
                phiDif[s,:] = phi_r[idD[s,:]] - phi[n,s]
                r_rs[s,:]  = r_r[idD[s,:]]
                
            kSr      = kS * r_rs
            sumRsp   = np.sum( kSr * np.cos( phiDif ), axis=1)
            sumPHIsp = np.sum( kSr * np.sin( phiDif ), axis=1)
                
            rdt         = rsum1 + rpro1 * sumRsp
            phidt       = omga + phipro1 * sumPHIsp
            # add differntial step
            r[n+1,:]    = r[n,:] + dt*rdt
            phi[n+1,:]  = np.remainder(phi[n,:] + dt*phidt, pi2)
        r       = r[maxDlay+1:,:]     # remove history samples used in the begining
        phi     = phi[maxDlay+1:,:]
        # simple downsampling (there may be aliasing)
        r   = r[::np.int32(1./(fs*dt)),:] 
        phi = phi[::np.int32(1./(fs*dt)),:]
        return r * np.exp(1j* phi)
    
    #numba.complex64[:,:](numba.float32[:,:],numba.float32[:,:],numba.int32,numba.int64[:,:],numba.float32,numba.float32,numba.float32,numba.float32,numba.float32[:,:],numba.float32) 
    #@numba.jit(nopython=True,cache=True,nogil=True,parallel=True,fastmath=True)
    def _KMAOcommu4(phi,r,maxDlay,dlayStep,eta,dlt,fs,dt,kS,omga,d_sin,d_cos):         
        C       = phi.shape[1]  
        
        idPhi  = np.int32(0)
        #idPhi  = np.zeros(C,dtype=np.int32)
        #kSr    = np.zeros((C))
        
        
        #nodes   = range(0,C)
        #commuOff = np.arange(0,C) * phi.shape[1]
        pi2     = np.float32(2 * np.pi)
        eta2    = np.float32(0.5 * eta)
        NoverPi2= np.float32(d_cos.shape[0] / pi2)
        sumRsp  = np.zeros((C, C),dtype=np.float32)
        sumPHIsp= np.zeros((C, C),dtype=np.float32)
        for n in range(maxDlay,phi.shape[0]-1):
            rsum1       = -dlt * r[n,:]
            rpro1       = eta2 * ( 1 - r[n,:] * r[n,:])# r*r
            phipro1     = eta2 * (r[n,:] + 1 / r[n,:]) # (r**2 + 1) / r
            idD = n - dlayStep
            #for s in nodes:
            for s in numba.prange(C):
                for i in range(C):
                    phiDif      = phi[idD[i,s],s] - phi[n,s]
                    kSr         = kS[i,s] * r[idD[i,s],s]
                    idPhi       = np.int32(phiDif * NoverPi2)
                    sumRsp[s,i]   = kSr * d_cos[idPhi]
                    sumPHIsp[s,i] = kSr * d_sin[idPhi]

            sumRsp_   = np.sum(sumRsp, axis=1)
            sumPHIsp_ = np.sum(sumPHIsp, axis=1)
            
            rdt         = rsum1 + rpro1 * sumRsp_
            phidt       = omga + phipro1 * sumPHIsp_
            # add differntial step
            r[n+1,:]    = r[n,:] + dt*rdt
            phi[n+1,:]  = np.remainder(phi[n,:] + dt*phidt, pi2)
        r       = r[maxDlay+1:,:]     # remove history samples used in the begining
        phi     = phi[maxDlay+1:,:]
        # simple downsampling (there may be aliasing)
        r   = r[::np.int64(1./(fs*dt)),:] 
        phi = phi[::np.int64(1./(fs*dt)),:]
        return r * np.exp(1j* phi)
    
    #numba.complex64[:,:](numba.float32[:,:],numba.float32[:,:],numba.int32,numba.int64[:,:],numba.float32,numba.float32,numba.float32,numba.float32,numba.float32[:,:],numba.float32) 
    @numba.jit(nopython=True,cache=True,nogil=True,parallel=True,fastmath=False)
    def _KMAOcommu(phi,r,maxDlay,dlayStep,eta,dlt,fs,dt,kS,omga):
        C       = phi.shape[1]
        #nodes   = range(0,C)
        #commuOff = np.arange(0,C) * phi.shape[1]
        pi2     = np.float64(2 * np.pi)
        eta2    = np.float64(0.5 * eta)
        sumRsp  = np.empty((C),dtype=np.float64)
        sumPHIsp= np.empty((C),dtype=np.float64)
        for n in range(maxDlay,phi.shape[0]-1):
            rsum1       = -dlt * r[n,:]
            rpro1       = eta2 * ( 1 - r[n,:] * r[n,:])# r*r
            phipro1     = eta2 * (r[n,:]  + 1 / r[n,:])
            
            idD = n * C - dlayStep
            
            phi_r  = phi.ravel()
            r_r    = r.ravel() 
            #import pdb; pdb.set_trace()
            #if n >= maxDlay*2 - 1:
            #    import pdb; pdb.set_trace()
            for s in range(C):
                phiDif      = phi_r[idD[s,:]] - phi[n,s]
                kSr         = kS[:,s] * r_r[idD[s,:]]  # good kS[:,s]
                sumRsp[s]   = np.sum( kSr * np.cos( phiDif ))
                sumPHIsp[s] = np.sum( kSr * np.sin( phiDif ))
            rdt         = rsum1 + rpro1 * sumRsp
            phidt       = omga + phipro1 * sumPHIsp
            # add differntial step
            r[n+1,:]    = r[n,:] + dt*rdt
            phi_ = phi[n,:] + dt*phidt
            phi[n+1,:]  = np.remainder(phi_, pi2)
        r       = r[maxDlay+1:,:]     # remove history samples used in the begining
        phi     = phi[maxDlay+1:,:]
        # simple downsampling (there may be aliasing)
        r   = r[::np.int32(1./(fs*dt)),:] 
        phi = phi[::np.int32(1./(fs*dt)),:]
        return (r * np.exp(1j* phi)).astype(np.complex64)
    
    def _doNodeContainers(self,maxDlay):      
        # node's variables
        #import pdb; pdb.set_trace()
        r           = np.zeros((int(self.tMax/self.dt + maxDlay), self.C) ,dtype=np.float64) # node phase parameter [C, Nsamples to integrate]
        phi         = np.zeros((int(self.tMax/self.dt + maxDlay), self.C) ,dtype=np.float64) # node phase parameter [C, Nsamples to integrate]
        
        # initial conditions as history for the time delays
        #omegaT      = self.omega * np.linspace(0,maxDlay*self.dt+self.dt,maxDlay+1)
        r[0:maxDlay+1,:]    = 0.3 * np.ones((maxDlay+1,self.C))
        #phi[0:maxDlay+1,:]  = np.tile(np.remainder(omegaT,2*np.pi), (self.C,1)).T
        phi[0:maxDlay+1,:]  = 0.3 * np.ones((maxDlay+1,self.C))
        #print(r.dtype, phi.dtype)
        return r, phi
    
    
    def setVariablesModel(self,x):
        "set the basic variable need to run the KM model"
        vel     = x[0]
        kL      = x[1]
        kG      = x[2]
        self.kS      = self.getAnatoCoupling(kG,kL)
        self.dlayStep, self.maxDlay = self.getDelays(vel)
        self.r, self.phi   = self._doNodeContainers(self.maxDlay)
        self.dlayId  = self.doIndexDelay(self.phi,self.dlayStep)
        #self.doSinCosIx()

/home/oscar/anaconda2/envs/ipy36/lib/python3.6/site-packages/numba/decorators.py:146: RuntimeWarning: Caching is not available when the 'parallel' target is in use. Caching is now being disabled to allow execution to continue.
  warnings.warn(msg, RuntimeWarning)


In [16]:
kmCol = TestkaoCol()
kmCol.fitness([0.9,150, 1200])

array([0.03906853])

In [7]:
kmCol2 = TestkaoCol()
kmCol2.fitness([0.9,150, 1200])

array([0.08003368])

In [22]:
kao = Testkao()
kao.fitness([0.9,150,1200])

array([0.10782875])

In [17]:
pl.figure()
pl.plot(np.abs(np.mean(kmCol.z, axis=1)))
pl.plot(np.mean(np.abs(kmCol.z), axis=1),'--')

In [8]:
pl.figure()
pl.plot(np.abs(np.mean(kmCol2.z, axis=1)))
pl.plot(np.mean(np.abs(kmCol2.z), axis=1),'--')

In [23]:
pl.figure()
pl.plot(np.abs(np.mean(kao.z, axis=0)))
pl.plot(np.mean(np.abs(kao.z), axis=0),'--')

In [106]:
#kmCol2 = TestkaoCol()
#kmCol.setVariablesModel([0.9,150, 1200])
#coef, corr = kmCol._fitFilterBands(kmCol.z.T)
#pl.plot(np.imag(zm))
pl.plot(np.abs(np.mean(kmCol.z, axis=1)))
pl.plot(np.mean(np.abs(kmCol.z), axis=1),'--')

In [17]:
kmCol3 = TestkaoCol()
kmCol3.setVariablesModel([0.9,150, 1200])
#kmRow = Testkao()
#kmRow.setVariablesModel([0.9,150, 1200])

In [24]:
print(np.sum(kmCol3.dlayStep[10,:]))

75132


In [20]:
np.remainder(np.arange(1,6,0.5),2*np.pi)

array([1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5])

In [68]:
%timeit kmCol._fitFilterBands(kmCol.z.T)
%timeit kmCol._fitFilterBands2(kmCol.z)

112 ms ± 3.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
130 ms ± 9.02 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%timeit TestkaoCol._KMAOcommu4(kmCol.phi,kmCol.r,kmCol.maxDlay,kmCol.dlayStep,kmCol.eta,kmCol.dlt,kmCol.fs,kmCol.dt,kmCol.kS,kmCol.omega,kmCol.d_sin,kmCol.d_cos)
%timeit Testkao._KMAOcommu4(kmRow.phi,kmRow.r,kmRow.maxDlay,kmRow.dlayStep,kmRow.eta,kmRow.dlt,kmRow.fs,kmRow.dt,kmRow.kS,kmRow.omega,kmRow.d_sin,kmRow.d_cos)

225 ms ± 744 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
277 ms ± 1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%timeit TestkaoCol._KMAOcommu(kmCol.phi,kmCol.r,kmCol.maxDlay,kmCol.dlayId,kmCol.eta,kmCol.dlt,kmCol.fs,kmCol.dt,kmCol.kS,kmCol.omega)
%timeit Testkao._KMAOcommu(kmRow.phi,kmRow.r,kmRow.maxDlay,kmRow.dlayId,kmRow.eta,kmRow.dlt,kmRow.fs,kmRow.dt,kmRow.kS,kmRow.omega)

1.83 s ± 4.91 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
275 ms ± 4.98 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [219]:
%timeit TestkaoCol._KMAOcommu2(kmCol.phi,kmCol.r,kmCol.maxDlay,kmCol.dlayId,kmCol.eta,kmCol.dlt,kmCol.fs,kmCol.dt,kmCol.kS,kmCol.omega)
%timeit Testkao._KMAOcommu2(kmRow.phi,kmRow.r,kmRow.maxDlay,kmRow.dlayId,kmRow.eta,kmRow.dlt,kmRow.fs,kmRow.dt,kmRow.kS,kmRow.omega)

4.52 s ± 183 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
4.58 s ± 148 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
%lprun -f TestkaoCol._KMAOcommu TestkaoCol._KMAOcommu(kmCol.phi,kmCol.r,kmCol.maxDlay,kmCol.dlay,kmCol.eta,kmCol.dlt,kmCol.fs,kmCol.dt,kmCol.kS,kmCol.omega)

TypeError: not enough arguments: expected 12, got 10

In [119]:
%lprun -f Testkao._KMAOcommu Testkao._KMAOcommu(kmRow.phi,kmRow.r,kmRow.maxDlay,kmRow.dlayId,kmRow.eta,kmRow.dlt,kmRow.fs,kmRow.dt,kmRow.kS,kmRow.omega)

In [26]:
%lprun -f TestkaoCol._KMAOcommu4 TestkaoCol._KMAOcommu4(kmCol.phi,kmCol.r,kmCol.maxDlay,kmCol.dlayStep,kmCol.eta,kmCol.dlt,kmCol.fs,kmCol.dt,kmCol.kS,kmCol.omega,kmCol.d_sin,kmCol.d_cos)